# Distributional semantics

Mehdi Ghanimifard, Adam Ek, Wafia Adouane and Simon Dobnik

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the instructions on how to work on group assignments.

Write all your answers and the code in the appropriate boxes below.

---

In this lab we will look how to build distributional semantic models from corpora and use semantic similarity captured by these models to do some simple semantic tasks. We are going to use the code that we discussed in the class last time.

The following command simply imports all the methods from that code.

In [ ]:
from dist_erk import *

## 1. Loading a corpus

To train a distributional model, we first need a sufficiently large collection of text which will contain different words used frequently enough in different contexts. Here we will use a section of the Wikipedia corpus which you can download from [here](https://linux.dobnik.net/oc/index.php/s/9NTlpOJfPWGS56t/download?path=%2Flab4-distributional-data&files=wikipedia.txt.zip) (Linux and Mac) or [here](https://linux.dobnik.net/oc/index.php/s/9NTlpOJfPWGS56t/download?path=%2Flab4-distributional-data&files=wikipedia-for-windows.zip) (Windows). (This file has been borrowed from another lab by [Richard Johansson](http://www.cse.chalmers.se/~richajo/).) When unpacked the file is 151mb hence if you are using the lab computers you should store it in a temporary folder outside your home and adjust `corpus_dir` path below.
<It may already exist in `/opt/mlt/courses/cl2015/a5`.>


In [ ]:
corpus_dir = 'wikipedia/'

## 2.1 Building a count-based model

Now you are ready to build a count-based model. The functions for building word spaces can be found in `dist_erk.py`. We will build a model that create count-based vectors for 1000 words. Using the methods from the code imported above build three word matrices with 1000 dimensions as follows: (i) with raw counts (saved to a variable `space_1k`); (ii) with PPMI (`ppmispace_1k`); and (iii) with reduced dimensions SVD (`svdspace_1k`). For the latter use `svddim=5`. **[5 marks]**


In [ ]:
word_to_keep = 1000
num_dims = 1000
svddim = 5

# which words to use as targets and context words?
ktw = ...

wi = ... # word index
words_in_order = ... # sorted words

print('create count matrices')
space_1k = ...
print('ppmi transform')
ppmispace_1k = ...
print('svd transform')
svdspace_1k = ...
print('done.')

## 2.2 Bulding a word2vec model

We will also build a continuous-bag-of-words (CBOW) word2vec model using gensim (https://radimrehurek.com/gensim/index.html). Build a CBOW word2vec model, where each word have 300 dimensions and as above, limit the vocabulary size to the most common 1000 words. **[5 marks]**

Documentation for the Word2Vec class can be found here: https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec

In [ ]:
from gensim import utils
from gensim.models import Word2Vec

# gensim require a iterable class to process the corpus
class CorpusReader():
    def __init__(self, corpus_path):
        self.corpus_path = corpus_path

    def __iter__(self):
        for line in open(self.corpus_path):
            sentence = utils.simple_preprocess(line)
            if sentence:
                yield sentence 
            
corpus = CorpusReader(corpus_dir+'wikipedia.txt')
w2v_model = Word2Vec(sentences=corpus,
                     # training options goes here
                     ...)

In [ ]:
print('house:', space_1k['house'])
print('house:', w2v_model['house'])

Oxford Advanced Dictionary has 185,000 words, hence 1,000 words is not representative. We trained a model with 10,000 words, and 50 dimensions on truncated SVD. It took 40 minutes on a laptop. 

Additionally, we trained a word2vec model on the same data. The vocabulary size is also 10,000 with 300 dimensions for the words, and truncated to 50 dimensions in SVD. It took about 15 minutes on a desktop.

We saved all five matrices [here](https://gubox.app.box.com/folder/75208243314) ([alternative/old link](https://linux.dobnik.net/oc/index.php/s/9NTlpOJfPWGS56t/download?path=%2Flab4-distributional-data&files=pretrained.zip)) which you can load as follows:

In [ ]:
import numpy as np

numdims = 10000
svddim = 50

print('Please wait..')
ktw_10k       = np.load('pretrained/ktw_wikipediaktw.npy', allow_pickle=True)
space_10k     = np.load('pretrained/raw_wikipediaktw.npy', allow_pickle=True).all()
ppmispace_10k = np.load('pretrained/ppmi_wikipediaktw.npy', allow_pickle=True).all()
svdspace_10k  = np.load('pretrained/svd50_wikipedia10k.npy', allow_pickle=True).all()
w2v_space     = np.load('pretrained/w2v.npy', allow_pickle=True)
w2v_svd_space = np.load('pretrained/w2v_svd.npy', allow_pickle=True)
print('Done.')


Each vector space can be queried as a dictionary $\texttt{(word_form: vector, ...)}$:

In [1]:
print('house:', space_10k['house'])
print('house:', w2v_space['house'])

NameError: name 'space_10k' is not defined

## 3. Operations on similarities

We can perform mathematical operations on word vectors to derive meaning predictions. For example, we can subtract the normalised vectors for `king` minus `queen` and add the resulting vector to `man` and we hope to get the vector for `woman`. Why? **[3 marks]**

**Your answer should go here:**

In [ ]:
# Your code should go here...

Here is some helpful code that allow us to calculate such comparisons.

In [ ]:
from scipy.spatial import distance

def normalize(vec):
    return vec / veclen(vec)

def find_similar_to(vec1, space):
    # vector similarity functions
    #sim_fn = lambda a, b: 1-distance.euclidean(normalize(a), normalize(b))
    #sim_fn = lambda a, b: 1-distance.correlation(a, b)
    #sim_fn = lambda a, b: 1-distance.cityblock(normalize(a), normalize(b))
    #sim_fn = lambda a, b: 1-distance.chebyshev(normalize(a), normalize(b))
    #sim_fn = lambda a, b: np.dot(normalize(a), normalize(b))
    sim_fn = lambda a, b: 1-distance.cosine(a, b)

    sims = [
        (word2, sim_fn(vec1, space[word2]))
        for word2 in space.keys()
    ]
    return sorted(sims, key = lambda p:p[1], reverse=True)

Here is how you apply this code. Compare the count-based method with the word2vec method and comment on the results you get. **[4 marks]**

In [ ]:
king = normalize(w2v_space['king'])
queen = normalize(w2v_space['queen'])
man = normalize(w2v_space['man'])
woman = normalize(w2v_space['woman'])

find_similar_to(king - man + woman, ...)[:10]

**Your answer should go here:**

Find 2 similar pairs of pairs of words and test them. Does the resulting vector similarity confirm your expectations? But remember you can only do this if the words are contained in our vector space. **[2 marks]**

**Your answer should go here:**

Try changing the number of dimensions, and the window size in the models you built in (2.1) and (2.2). Comment on the new results you get in comparison to the first results results. [**4 marks**]

**Your answer should go here:**

## 4. Testing semantic similarity

The file `similarity_judgements.txt` (a copy is included with this notebook or you can download it from [here](https://linux.dobnik.net/oc/index.php/s/9NTlpOJfPWGS56t/download?path=%2Flab4-distributional-data&files=similarity_judgements.txt.zip)) contains 7,576 pairs of words and their lexical and visual similarities (based on the pictures) collected in on online crowd-sourcing data collection using Mechanical Turk as described in [1]. The score range from 1 (highly dissimilar) to 5 (highly similar).

The following code will import them into python lists below:

In [ ]:
word_pairs = [] # test suit word pairs
semantic_similarity = [] 
visual_similarity = []
test_vocab = set()

for index, line in enumerate(open('similarity_judgements.txt')):
    data = line.strip().split('\t')
    if index > 0 and len(data) == 3:
        w1, w2 = tuple(data[0].split('#'))
        # it will check if both words from each pair exist in the word matrix.
        if w1 in space_1k and w2 in space_1k:
            word_pairs.append((w1, w2))
            test_vocab.update([w1, w2])
            semantic_similarity.append(float(data[1]))
            visual_similarity.append(float(data[2]))
        
print("number of available words to test:", len(test_vocab-(test_vocab-set(ktw))))
print("number of available word pairs to test:", len(word_pairs))
#list(zip(word_pairs, visual_similarity, semantic_similarity))

Now we can test how the cosine similarity between vectors of each of the five spaces compares with the human judgements on the words collected in the previous step. Which of the five spaces best approximates human judgements?

For comparison of several scores we can use [Spearman correlation coefficient](https://en.wikipedia.org/wiki/Spearman's_rank_correlation_coefficient) which is implemented in `scipy.stats.spearmanr` [here](https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.stats.spearmanr.html). The values of the Sperman correlation coefficient range from -1, 0 to 1, where 0 indicates no correlation, 1 perfect correaltion and -1 negative correlation. Hence, the greater the number the better. The $p$-values tells us if the coefficient is statistically significant. For this to be the case, it must be less than or equal to $< 0.05$.

Here is how you can calculate Spearman's correlation coefficient betweeen the scores of visual similarity and semantic similarity of the available words in the test suite:

In [ ]:
from scipy import stats

rho, pval = stats.spearmanr(semantic_similarity, visual_similarity)
print("""Visual Similarity vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))


Let's now calculate the cosine similarity scores of all word pairs in an ordered list using all three matrices. **[2 marks]**

In [ ]:
raw_similarities  = ["code here" for w1, w2 in word_pairs]
ppmi_similarities = ["code here" for w1, w2 in word_pairs]
svd_similarities  = ["code here" for w1, w2 in word_pairs]

w2v_similarities     = ["code here" for w1, w2 in word_pairs]
w2v_svd_similarities = ["code here" for w1, w2 in word_pairs]

Now, calculate correlation coefficients between lists of similarity scores and the real semantic similarity scores from the experiment. The scores of what model best correlate them? Is this expected? **[4 marks + 2 marks]**

In [ ]:
# Your code should go here...

**Your answer should go here:***

We can also calculate correlation coefficients between lists of cosine similarity scores and the real visual similarity scores from the experiment. Which similarity model best correlates with them? How do the correlation coefficients compare with those from the previous comparison - and can you speculate why do we get such results? **[2 marks + 6 marks]**

In [ ]:
# Your code should go here...

**Your answer should go here:**

## 5. Discussion

What are the limitations of our approach in this lab? Suggest three ways in which the results could be improved. **[6 marks]**

**Your answer should go here:**

# Literature

[1] C. Silberer and M. Lapata. Learning grounded meaning representations with autoencoders. In Proceedings of the 52nd Annual Meeting of the Association for Computational Linguistics, pages 721–732, Baltimore, Maryland, USA, June 23–25 2014 2014. Association for Computational Linguistics.

[2] Y. Bengio, R. Ducharme, P. Vincent, & C. Jauvin (2003). A neural probabilistic language model. Journal of machine learning research, 3(Feb), 1137-1155.

[3] Levy, Omer, Yoav Goldberg, and Ido Dagan. "Improving distributional similarity with lessons learned from word embeddings." Transactions of the Association for Computational Linguistics 3 (2015): 211-225.
